In [1]:
import json 
import os
import io
import collections
import math
import random
import six
from PIL import Image

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from datetime import datetime

from pprint import pprint
from shutil import copyfile, move

import keras
import resnet
import keras.backend as K
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D, Conv2D
from keras.optimizers import SGD, Adam
from keras.layers.merge import add
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2

%matplotlib inline

Using TensorFlow backend.


In [2]:
WIDTH = 224
HEIGHT = 224
as_grey = False
batch_size = 16
test_batch_size = 16
epochs = 500

In [3]:
from __future__ import division

import six
from keras.models import Model
from keras.layers import (
    Input,
    Activation,
    Dense,
    Flatten
)
from keras.layers.convolutional import (
    Conv2D,
    MaxPooling2D,
    AveragePooling2D
)
from keras.layers.merge import add
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K


def _bn_relu(input):
    """Helper to build a BN -> relu block
    """
    norm = BatchNormalization(axis=CHANNEL_AXIS)(input)
    return Activation("relu")(norm)


def _conv_bn_relu(**conv_params):
    """Helper to build a conv -> BN -> relu block
    """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        conv = Conv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(input)
        return _bn_relu(conv)

    return f


def _bn_relu_conv(**conv_params):
    """Helper to build a BN -> relu -> conv block.
    This is an improved scheme proposed in http://arxiv.org/pdf/1603.05027v2.pdf
    """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        activation = _bn_relu(input)
        return Conv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(activation)

    return f


def _shortcut(input, residual):
    """Adds a shortcut between input and residual block and merges them with "sum"
    """
    # Expand channels of shortcut to match residual.
    # Stride appropriately to match residual (width, height)
    # Should be int if network architecture is correctly configured.
    input_shape = K.int_shape(input)
    residual_shape = K.int_shape(residual)
    stride_width = int(round(input_shape[ROW_AXIS] / residual_shape[ROW_AXIS]))
    stride_height = int(round(input_shape[COL_AXIS] / residual_shape[COL_AXIS]))
    equal_channels = input_shape[CHANNEL_AXIS] == residual_shape[CHANNEL_AXIS]

    shortcut = input
    # 1 X 1 conv if shape is different. Else identity.
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        shortcut = Conv2D(filters=residual_shape[CHANNEL_AXIS],
                          kernel_size=(1, 1),
                          strides=(stride_width, stride_height),
                          padding="valid",
                          kernel_initializer="he_normal",
                          kernel_regularizer=l2(0.0001))(input)

    return add([shortcut, residual])


def _residual_block(block_function, filters, repetitions, is_first_layer=False):
    """Builds a residual block with repeating bottleneck blocks.
    """
    def f(input):
        for i in range(repetitions):
            init_strides = (1, 1)
            if i == 0 and not is_first_layer:
                init_strides = (2, 2)
            input = block_function(filters=filters, init_strides=init_strides,
                                   is_first_block_of_first_layer=(is_first_layer and i == 0))(input)
        return input

    return f


def basic_block(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    """Basic 3 X 3 convolution blocks for use on resnets with layers <= 34.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf
    """
    def f(input):

        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv1 = Conv2D(filters=filters, kernel_size=(3, 3),
                           strides=init_strides,
                           padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=l2(1e-4))(input)
        else:
            conv1 = _bn_relu_conv(filters=filters, kernel_size=(3, 3),
                                  strides=init_strides)(input)

        residual = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv1)
        return _shortcut(input, residual)

    return f


def bottleneck(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    """Bottleneck architecture for > 34 layer resnet.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf

    Returns:
        A final conv layer of filters * 4
    """
    def f(input):

        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv_1_1 = Conv2D(filters=filters, kernel_size=(1, 1),
                              strides=init_strides,
                              padding="same",
                              kernel_initializer="he_normal",
                              kernel_regularizer=l2(1e-4))(input)
        else:
            conv_1_1 = _bn_relu_conv(filters=filters, kernel_size=(1, 1),
                                     strides=init_strides)(input)

        conv_3_3 = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv_1_1)
        residual = _bn_relu_conv(filters=filters * 4, kernel_size=(1, 1))(conv_3_3)
        return _shortcut(input, residual)

    return f


def _handle_dim_ordering():
    global ROW_AXIS
    global COL_AXIS
    global CHANNEL_AXIS
    if K.image_dim_ordering() == 'tf':
        ROW_AXIS = 1
        COL_AXIS = 2
        CHANNEL_AXIS = 3
    else:
        CHANNEL_AXIS = 1
        ROW_AXIS = 2
        COL_AXIS = 3


def _get_block(identifier):
    if isinstance(identifier, six.string_types):
        res = globals().get(identifier)
        if not res:
            raise ValueError('Invalid {}'.format(identifier))
        return res
    return identifier


class ResnetBuilder(object):
    @staticmethod
    def build(input_shape, num_outputs, block_fn, repetitions):
        """Builds a custom ResNet like architecture.

        Args:
            input_shape: The input shape in the form (nb_channels, nb_rows, nb_cols)
            num_outputs: The number of outputs at final softmax layer
            block_fn: The block function to use. This is either `basic_block` or `bottleneck`.
                The original paper used basic_block for layers < 50
            repetitions: Number of repetitions of various block units.
                At each block unit, the number of filters are doubled and the input size is halved

        Returns:
            The keras `Model`.
        """
        _handle_dim_ordering()
        if len(input_shape) != 3:
            raise Exception("Input shape should be a tuple (nb_channels, nb_rows, nb_cols)")

        # Permute dimension order if necessary
        if K.image_dim_ordering() == 'tf':
            input_shape = (input_shape[1], input_shape[2], input_shape[0])

        # Load function from str if needed.
        block_fn = _get_block(block_fn)

        input = Input(shape=input_shape)
        conv1 = _conv_bn_relu(filters=64, kernel_size=(7, 7), strides=(2, 2))(input)
        pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(conv1)

        block = pool1
        filters = 64
        for i, r in enumerate(repetitions):
            block = _residual_block(block_fn, filters=filters, repetitions=r, is_first_layer=(i == 0))(block)
            filters *= 2

        # Last activation
        block = _bn_relu(block)

        # Classifier block
        block_shape = K.int_shape(block)
        pool2 = AveragePooling2D(pool_size=(block_shape[ROW_AXIS], block_shape[COL_AXIS]),
                                 strides=(1, 1))(block)
        flatten1 = Flatten()(pool2)
        dense = Dense(units=num_outputs, kernel_initializer="he_normal",
                      activation="softmax")(flatten1)

        model = Model(inputs=input, outputs=dense)
        return model

    @staticmethod
    def build_resnet_18(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [2, 2, 2, 2])

    @staticmethod
    def build_resnet_34(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [3, 4, 6, 3])

    @staticmethod
    def build_resnet_50(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 6, 3])

    @staticmethod
    def build_resnet_101(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 23, 3])

    @staticmethod
    def build_resnet_152(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 8, 36, 3])

In [4]:
# load labels
# train_dict = {}
# train_labels=open(annotation_path_train)
# data_train = json.load(train_labels)
# for elem in data_train['annotations']:
#     train_dict[elem['image_id']] = elem['category_id']

# train_labels.close()
# training_size = len(train_dict.values())

# val_dict = {}
# val_labels=open(annotation_path_val)
# data_val = json.load(val_labels)
# for elem in data_val['annotations']:
#     val_dict[elem['image_id']] = elem['category_id']    

# val_labels.close()
# val_size = len(val_dict.values())

# for key, value in train_dict.items():
#     copyfile(os.getcwd() + '/data/train_val/{}.jpg'.format(key), os.getcwd() + '/data/train/{}/{}.jpg'.format(value, key))
    
# for key, value in val_dict.items():
#     copyfile(os.getcwd() + '/data/train_val/{}.jpg'.format(key), os.getcwd() + '/data/val/{}/{}.jpg'.format(value, key))

In [5]:
# load images with generators

# train_path = os.getcwd() + '/data/Clouded_leopard_ID/train_cleaned_cropped'
# test_path = os.getcwd() + '/data/Clouded_leopard_ID/test_cleaned_cropped'

train_path = 'D://Species//train_cropped'
test_path = 'D://Species//test_cropped'


images = []

# augmentation configuration for training
train_datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rescale=1./255,
        shear_range=0.1,
#         zoom_range=0.25,
        horizontal_flip=True,
        vertical_flip=True)

# augmentation configuration for testing:
validation_datagen = ImageDataGenerator(
#         rotation_range=90,
#         width_shift_range=0.25,
#         height_shift_range=0.25,
        rescale=1./255,
#         shear_range=0.25,
#         zoom_range=0.25,
#         horizontal_flip=True,
#         vertical_flip=True
)

# train batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(HEIGHT, WIDTH),
        batch_size=batch_size,
        class_mode='categorical')

# this is a similar generator, for validation data
validation_generator = validation_datagen.flow_from_directory(
        test_path,
        target_size=(HEIGHT, WIDTH),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

unique, counts = np.unique(train_generator.classes, return_counts=True)
class_weights = dict(zip(unique, counts))

Found 408979 images belonging to 87 classes.
Found 102288 images belonging to 87 classes.


In [6]:
def visualize(history):
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [7]:
def save_pred(pred):
    ids = [file[4:-4] for file in test_generator.filenames]
    df = pd.DataFrame(np.append(ids, pred).reshape(2, len(pred)).T)
    df.columns = ['id','animal_present']
    df.to_csv('./predictions2.csv', index=False)

In [8]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
K.set_session(session)

In [14]:
run_name = 'scratch_inital_res50'
# compute with exiting model for 29 classes

# input_tensor = Input(shape=(WIDTH, HEIGHT, 3))

# # VGG without FC layers
# # model = keras.applications.vgg16.VGG16(include_top=False, weights="imagenet")
# model = keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet',
#                                      input_tensor=input_tensor, classes=2)
# x = model(input_tensor)

training_size = len(train_generator.filenames)
validation_size = len(validation_generator.filenames)

# model = keras.models.load_model(os.getcwd() + '/models/cloud_June 18, 2018.hdf5')
model = ResnetBuilder.build_resnet_50((3, HEIGHT, WIDTH), 87)
K.set_image_dim_ordering('tf')

# Classification block
# x = Flatten(name='flatten')(x)
# x = Dense(512, activation='relu', name='fc1')(x)
# x = Dense(1024, activation='relu', name='fc2')(x)
# x = Dense(29, activation='softmax', name='predictions')(x)
# model = Model(input=input_layer, output=x)
    
# set up tensorboard for progress and analysis 
tensorboard = TensorBoard(log_dir='D:\\Species_Graphs/Wednesday_9_19_2018_{}'.format(run_name), histogram_freq=0,
                                          batch_size=batch_size, write_graph=False, write_grads=False,
                                          write_images=True)
# save model after each epoch
root = 'D:\\Species_Snapshots/Wednesday_9_19_2018_{}'.format(run_name)
if not os.path.isdir(root):
    os.mkdir(root)
file_path = root + '/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath=file_path, monitor='val_acc', verbose=1, save_best_only=False)

# reduce lr on plateau of val loss
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0)

adam = Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

train_steps = math.ceil(training_size / batch_size)
val_steps = math.ceil(validation_size / batch_size)

# train_steps = 1
# val_steps = 1

history = model.fit_generator(train_generator, steps_per_epoch=train_steps, epochs=epochs,
                    validation_data=validation_generator, validation_steps=val_steps, verbose=1,
                              callbacks=[tensorboard, checkpoint, reduce_lr],
#                               class_weight=class_weights, 
                             )

# model.save(os.getcwd() + '/models/cloud_custom_{}.hdf5'.format(datetime.now().strftime("%B %d, %Y")))
visualize(history)

Epoch 1/500
25562/25562 [==============================] - 34085s 1s/step - loss: 6.0124 - acc: 0.5605 - val_loss: 5.2391 - val_acc: 0.6274

Epoch 00001: saving model to D:\Species_Snapshots/Wednesday_9_19_2018_scratch_inital_res50/weights-improvement-01-0.63.hdf5
Epoch 2/500
25562/25562 [==============================] - 35161s 1s/step - loss: 4.7629 - acc: 0.6721 - val_loss: 4.4953 - val_acc: 0.6789

Epoch 00002: saving model to D:\Species_Snapshots/Wednesday_9_19_2018_scratch_inital_res50/weights-improvement-02-0.68.hdf5
Epoch 3/500
25562/25562 [==============================] - 32029s 1s/step - loss: 4.1461 - acc: 0.7268 - val_loss: 4.0146 - val_acc: 0.7157

Epoch 00003: saving model to D:\Species_Snapshots/Wednesday_9_19_2018_scratch_inital_res50/weights-improvement-03-0.72.hdf5
Epoch 4/500
25562/25562 [==============================] - 32063s 1s/step - loss: 3.7479 - acc: 0.7576 - val_loss: 3.7593 - val_acc: 0.7265

Epoch 00004: saving model to D:\Species_Snapshots/Wednesday_9_19

KeyboardInterrupt: 

In [ ]:
pred = model.predict_generator(test_generator, len([name for name in os.listdir(os.getcwd() + '/data/test/all')]) // test_batch_size, verbose=1)
save_pred(pred.argmax(axis=-1))

In [16]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7880765986878094097
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9221160305
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4198711626723970719
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:65:00.0, compute capability: 6.1"
]
